In [48]:
import pandas as pd
import numpy as np
import regex as re

In [81]:
dataset = pd.read_csv("dataset/PlayerData.csv", low_memory=False, index_col=0)

In [82]:
dataset.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Acceleration,Aggression,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
Name,,,,,,,,,,,,,,,,,,,,,
Cristiano Ronaldo,32,Portugal,94,94,Real Madrid CF,â‚¬95.5M,â‚¬565K,2228,89,63,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
L. Messi,30,Argentina,93,93,FC Barcelona,â‚¬105M,â‚¬565K,2154,92,48,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
Neymar,25,Brazil,92,94,Paris Saint-Germain,â‚¬123M,â‚¬280K,2100,94,56,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
L. SuÃ¡rez,30,Uruguay,92,92,FC Barcelona,â‚¬97M,â‚¬510K,2291,88,78,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
M. Neuer,31,Germany,92,92,FC Bayern Munich,â‚¬61M,â‚¬230K,1493,58,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
def cleanData(val):
    if type(val) is str:
        val = re.sub(val,'[^a-zA-Z0-9]+','')
    return val

In [79]:
for col in dataset.columns:
    dataset[col] = dataset[col].apply(cleanData)

In [87]:
a = "â‚¬95.5M"

In [91]:
re.sub(a,'[^a-z^A-Z^0-9]','')

''

In [92]:
a

'â‚¬95.5M'